In [4]:
import os
import cv2
import mediapipe as mp
import pandas as pd

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

dataset_folder = "dataset"

output_folder = "dataset_vectors"
os.makedirs(output_folder, exist_ok=True)

landmark_names = mp_pose.PoseLandmark

for pose_folder in os.listdir(dataset_folder):
    pose_path = os.path.join(dataset_folder, pose_folder)

    if os.path.isdir(pose_path):
        output_subfolder = os.path.join(output_folder, pose_folder)
        os.makedirs(output_subfolder, exist_ok=True)

        images = [os.path.join(pose_path, img) for img in os.listdir(pose_path) if img.endswith(('.jpg', '.png'))]

        for img_path in images:
            image = cv2.imread(img_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if not results.pose_landmarks:
                continue  # Si no hay landmarks, saltar la imagen

            landmarks = {lm_idx: (lm.x, lm.y, lm.z) for lm_idx, lm in enumerate(results.pose_landmarks.landmark)}

            image_data = []

            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx, end_idx = connection

                if start_idx not in landmarks or end_idx not in landmarks:
                    continue

                x1, y1, z1 = landmarks[start_idx]
                x2, y2, z2 = landmarks[end_idx]

                vec_x = x2 - x1
                vec_y = y2 - y1
                vec_z = z2 - z1

                start_name = landmark_names(start_idx).name
                end_name = landmark_names(end_idx).name
                vector_name = f"{start_name}_{end_name}"

                image_data.append([vector_name, vec_x, vec_y, vec_z])

            # Crear el DataFrame
            df = pd.DataFrame(image_data, columns=["name", "x", "y", "z"])

            # Guardar CSV con el mismo nombre que la imagen
            img_name = os.path.splitext(os.path.basename(img_path))[0]
            output_csv_path = os.path.join(output_subfolder, f"{img_name}.csv")
            df.to_csv(output_csv_path, index=False)

# Cerrar MediaPipe Pose
pose.close()

print("CSV por imagen generados correctamente.")


CSV por imagen generados correctamente.
